1. Usage of the provided book codes:

See:

http://www.computational-semantics.eu/FSynF.hs

http://www.computational-semantics.eu/Model.hs

http://www.computational-semantics.eu/MCWPL.hs

http://www.computational-semantics.eu/TCOM.hs

Answer the following questions:

- In FSynF, looking at the definition of "data NP", you can see the author uses one constructor for each person name. Is there any advantage of disadvantage? Hint: compare this with your approach in HW2.
- Looking at the definition of "data Formula a", is there any different in semantics of different instances of "a"? Similarly, what is the semantics of different instances of "a" and "b" in this Tree definition (notice that it is not the definition in Data.Tree):

$data\ Tree\ a\ b = Leaf\ a\ |\ Branches\ b\ [Tree\ a\ b]$
- Compare between two model checking approaches in MCWPL and TCOM. What are the advantages and disadvantages of each? 

2. Usage of State and StateT Monad

Let's do the following problem:

A block locates at position (0,0) on a gridboard. You move it through a sequence of commands (left, left, up, up, right, right). Find its final position?

In [22]:
import Control.Monad
import Control.Monad.Trans.State

-- Insated of using constructor for StateT
-- You create states through function "state"
-- Inside, you wrap a function that inputs a state_value
-- and outputs (result, state_value)
-- In this example, we just ignore the result, and only
-- change state_value, which is the location of the block.

right, left, up, down :: Monad m => StateT (Int, Int) m ()
right = state (\(x,y) -> ((), (x+1,y))) 
left = state (\(x,y) -> ((), (x-1,y)))
up = state (\(x,y) -> ((), (x,y + 1)))
down = state (\(x,y) -> ((), (x,y - 1)))

-- do notation allows combining multiple states
commands = do { left; left; up; up; right; right } 

In [23]:
execState commands (0,0)

(0,2)

Let's the sequence of commands to be something like (5 left, 2 right, 3 up). Give the final position

In [24]:
commands = do { replicateM_ 5 left; replicateM_ 2 right; replicateM_ 3 up } 

In [25]:
execState commands (0,0)

(-3,3)

Let's print out all intermediate positions

In [26]:
import Control.Monad.IO.Class

-- A generic function to move with translocation of (a,b)
move :: (Int, Int) -> StateT (Int, Int) IO ()

move = \(a,b) -> do
   (x, y) <- get
   liftIO $ print (x,y) -- Notice the requirement of liftIO 
   put (x + a, y + b)
   return ()
   

In [27]:
rightP = move (1,0) 
leftP = move (-1,0) 
upP = move (0,1) 
downP = move (0,-1) 

In [28]:
-- Same commands 
commands = do { replicateM_ 5 leftP; replicateM_ 2 rightP; replicateM_ 3 upP }

In [30]:
execStateT commands (0,0) -- Notice the use of execStateT instead of execState

(0,0)
(-1,0)
(-2,0)
(-3,0)
(-4,0)
(-5,0)
(-4,0)
(-3,0)
(-3,1)
(-3,2)
(-3,3)

Answer the following questions:
- Find out why do we need function liftIO.
- Find out why do you have two different functions "execState" and "execStateT"?
- Think of a situation, when you need to add an independent state to your system, like when you want to change the color of your block among "red", "green" and "blue". Now how do you change your code so that you can receive a list of commands like (right, right, up, up, <b>blue</b>, up, <b>red</b>) and print out the new position and current color? Hint: you shouldn't change your code in the previous implementation of left, right, up, down. 